In [ ]:
%pip install -qqqU langchain-openai langgraph

In [ ]:
import os

from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain_aws import BedrockEmbeddings

embeddings = BedrockEmbeddings()

qdrant_client = QdrantClient(
    location=os.getenv("VECTOR_STORE_URL", "http://localhost:6333"),
    api_key=os.getenv("VECTOR_STORE_API_KEY", "api_key"),
)

vector_store = QdrantVectorStore(
    client=qdrant_client, collection_name="regulamento-semantic", embedding=embeddings
)